# Do the scoliosis EMG analysis for one patient

In [ ]:
import pathlib

import numpy as np
from collections import defaultdict

import pandas as pd

from gaitutils.stats import collect_trial_data
from gaitutils.trial import Trial
from gaitutils import eclipse
from gaitutils.config import cfg

import matplotlib.pyplot as plt
#%matplotlib widget

In [ ]:
#DATA_FLDR = 'Y:/Userdata_Vicon_Server/Skolioosi/Potilaina/E0145_AA/2022_02_10_Seur_AA'
#DATA_FLDR = 'Y:/Userdata_Vicon_Server/Skolioosi/Potilaina/E0173_GL/2023_02_16_skolioosi_seur_GL'
DATA_FLDR = 'Y:/Userdata_Vicon_Server/Skolioosi/Potilaina/E0148_AK/2022_04_11_Seur_AK'
#DATA_FLDR = 'Y:/Userdata_Vicon_Server/Skolioosi/Potilaina/E0150_EA/2022_05_05_seur_EA'
#DATA_FLDR = 'Y:/Userdata_Vicon_Server/Skolioosi/Potilaina/E0156_EO/2022_8_8_PreKorsetti'
#DATA_FLDR = 'Y:/Userdata_Vicon_Server/Skolioosi/Potilaina/E0163_IR/2022_11_17_skolioosi_IR'

WALKING_TAGS = {'E1', 'E2', 'E3', 'E4', 'T1', 'T2', 'T3', 'T4'}
MVC_TAG = 'MVC'

cfg.trial.no_toeoff = 'reject'

## Read all the walking files

In [ ]:
emg_walk = defaultdict(lambda: np.zeros((1000,0)))

for c3d_file in pathlib.Path(DATA_FLDR).glob('*.c3d'):
    print(f'Reading file {c3d_file} ...')
    trial = Trial(c3d_file)
    
    if trial.eclipse_tag in WALKING_TAGS:
        print(f'Collecting trial data for file {c3d_file} (walking) ...')
        data, cycles = collect_trial_data(trial, analog_envelope=True, force_collect_all_cycles=True, fp_cycles_only=False)
        for ch in data['emg'].keys():
            emg_walk[ch] = np.hstack((emg_walk[ch], data['emg'][ch].T))

## Read all the reference files

In [ ]:
emg_ref = defaultdict(lambda: np.zeros((0,)))

for c3d_file in pathlib.Path(DATA_FLDR).glob('*.c3d'):
    trial = Trial(c3d_file)
    
    if MVC_TAG in eclipse.get_eclipse_keys(trial.enfpath)['NOTES'] or MVC_TAG in eclipse.get_eclipse_keys(trial.enfpath)['DESCRIPTION']:
        print(f'Reading file {c3d_file} (MVC) ...')
        for ch in emg_walk.keys():
            emg_ref[ch] = np.concatenate((emg_ref[ch], trial.get_emg_data(ch, envelope=True)[1]))

## Normalize

In [ ]:
for ch in emg_walk:
    emg_walk[ch] /= emg_ref[ch].max()

## Plot

In [ ]:
ch_base = set(ch[1:] for ch in emg_walk.keys())

for ch in ch_base:
    plt.figure()
    
    plt.subplot(1, 2, 1)
    plt.plot(emg_walk[f'R{ch}'], color='green')
    plt.plot(emg_walk[f'L{ch}'], color='red')
    plt.title(f'{ch}')
    
    plt.subplot(1, 2, 2)
    plt.plot(emg_walk[f'R{ch}'].mean(axis=1), color='green')
    plt.plot(emg_walk[f'R{ch}'].mean(axis=1) + emg_walk[f'R{ch}'].std(axis=1), color='green', linestyle='dotted')
    plt.plot(emg_walk[f'R{ch}'].mean(axis=1) - emg_walk[f'R{ch}'].std(axis=1), color='green', linestyle='dotted')

    plt.plot(emg_walk[f'L{ch}'].mean(axis=1), color='red')
    plt.plot(emg_walk[f'L{ch}'].mean(axis=1) + emg_walk[f'L{ch}'].std(axis=1), color='red', linestyle='dotted')
    plt.plot(emg_walk[f'L{ch}'].mean(axis=1) - emg_walk[f'L{ch}'].std(axis=1), color='red', linestyle='dotted')

    plt.title(f'{ch} (avg + std)')
    plt.gcf().set_size_inches(20, 6)


## Compute the indices

In [ ]:
res = {'index': ('right max', 'std of (right max)', 'left max', 'std of (left max)', 'right average', 'left average')}
                 
for ch in ch_base:
    res[ch] = (np.mean(emg_walk[f'R{ch}'].max(axis=0)), np.std(emg_walk[f'R{ch}'].max(axis=0)), np.mean(emg_walk[f'L{ch}'].max(axis=0)), np.std(emg_walk[f'L{ch}'].max(axis=0)), emg_walk[f'R{ch}'].mean(), emg_walk[f'L{ch}'].mean())
    
df = pd.DataFrame(res)
df